<a href="https://colab.research.google.com/github/KrestNIGH/Ryasanov-Nikita/blob/master/ML_to_pipeline_2209.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Все ещё хуже

In [83]:
from google.colab import drive
import pandas as pd
import numpy as np
import copy
from sklearn.metrics import make_scorer
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import pickle

In [84]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [85]:
#в общем пайплайне будет не нужен
def our_eternal_errors(x, sorrows=False):
  if sorrows:
    x = x.split(', ')[1:]
    x[-1] = x[-1][0:-2]
  else:
    x = x.split(', ')
    x[-1] = x[-1][0:-1]
  x[0] = x[0][1:]
  return list(map(float, x)) 
def and_our_sorrows(x):
  return int(x.split(',')[0][1:])


In [86]:
def gauss_scorer_func(y_true, y_pred):
  y_true = y_true.astype('float64')
  gauss = np.exp(-5*((y_pred-y_true)/y_true)**2)
  return np.median(gauss)
gauss_scorer = make_scorer(gauss_scorer_func, greater_is_better=True)#scorer object

In [90]:
def array_cutting(array_1, size=40):
  array = copy.deepcopy(array_1)
  size_difference = len(array) - size
  if size_difference <= 0:
    return array_padding(array, size)
  while len(array) > size:
    if len(array) - size >= 2:
      array.pop(0)
      array.pop(-1)
    else:
      array.pop(0)
  return array

In [91]:
def array_padding(array_1, size, flag='value', value=1, only_up=True):
  array = copy.deepcopy(array_1)
  size_difference = size - len(array)
  if size_difference == 0:
    return array
  elif size_difference < 0:
    raise Exception('Array length exceeds given size')

  if flag == 'mean':
    value = np.mean(array)
  elif flag == 'median':
    value = np.median(array)
  elif flag == 'value':
    pass
  else:
    raise Exception('Incorrect flag')

  if only_up:
    while len(array) < size:
      array.insert(0, value)
  else:
    array.extend([value] * round((size_difference)//2))
    [array.insert(0, value) for i in range(round((size_difference)//2))]
    while len(array) < size:
      array.insert(0, value)

  return array

In [92]:
# uploading pre-trained algorithms
rf = pickle.load(open('/content/drive/My Drive/Dom_rf_project/search_rf.pkl', 'rb'))
svr = pickle.load(open('/content/drive/My Drive/Dom_rf_project/search_svr.pkl', 'rb'))
xgb1 = xgb.Booster()
xgb1.load_model('/content/drive/My Drive/Dom_rf_project/search_xgb.bin')
knr = pickle.load(open('/content/drive/My Drive/Dom_rf_project/search_knr.pkl', 'rb'))

[10:54:49] WARNING: ../src/learner.cc:764: Loading model from XGBoost < 1.0.0, consider saving it again for improved compatibility


In [103]:
"""
весь пайплайн должен получать на вход несколько картинок, соответсвующих одному id, и выдавать для него одно предсказание
на части с машинкой на вход идёт массив массивов корреляций, по каждому из массивов делается предсказание и на выходе нужно усреднять предсказания(возможно, учитывая выбросы)
input = cor_lists(list of cor_list)
"""
cor_lists = [array_cutting(our_eternal_errors(df_test.pred_cor_both[i], sorrows=False)) for i in range(3)]
amount = len(cor_lists)
cor_lists.append([1] * 40)
cor_df = pd.DataFrame()
for i in range(40):
  cor_df[f'padded_corrlations_{i}'] = [x[i] for x in cor_lists]

In [104]:
cor_df

,padded_corrlations_0,padded_corrlations_1,padded_corrlations_2,padded_corrlations_3,padded_corrlations_4,padded_corrlations_5,padded_corrlations_6,padded_corrlations_7,padded_corrlations_8,padded_corrlations_9,padded_corrlations_10,padded_corrlations_11,padded_corrlations_12,padded_corrlations_13,padded_corrlations_14,padded_corrlations_15,padded_corrlations_16,padded_corrlations_17,padded_corrlations_18,padded_corrlations_19,padded_corrlations_20,padded_corrlations_21,padded_corrlations_22,padded_corrlations_23,padded_corrlations_24,padded_corrlations_25,padded_corrlations_26,padded_corrlations_27,padded_corrlations_28,padded_corrlations_29,padded_corrlations_30,padded_corrlations_31,padded_corrlations_32,padded_corrlations_33,padded_corrlations_34,padded_corrlations_35,padded_corrlations_36,padded_corrlations_37,padded_corrlations_38,padded_corrlations_39
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.290862,0.659733,0.398386,0.093360,0.087235,0.085757,0.167856,0.311675,0.257201,0.201642,-0.349122,0.046703,-0.154900
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.231047,0.137848,-0.131317,0.300329,0.877377,0.858289,0.851380,0.891163,0.828916,0.839487,0.842051,0.690701,0.668176,0.528589,0.460643,0.467501,-0.080125,-0.085299,-0.176719,-0.143744,-0.768313
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.073135,0.185721,0.357965,0.206891,0.600238,0.729677,0.637508,0.745011,0.683721,0.621307,0.047662,0.405462,0.763117,0.708670,0.735394,0.680394,0.731078,0.668980,0.504781,0.279911,0.133213
3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [106]:
predictions = []
for i in range(amount):
  prediction = 0
  for regressor in (rf, svr, xgb1, knr):
    if regressor == xgb1:
      data = xgb.DMatrix(cor_df)
      prediction += regressor.predict(data)[i]
    else:
      prediction += regressor.predict(cor_df)[i]
  predictions.append(prediction/len(names))
output = np.median(predictions)
output

11.943222979110635